<a href="https://colab.research.google.com/github/MustafizSaadi/DMLab/blob/main/Decision_Tree_Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gzip
from io import StringIO
import re
import sys
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [ ]:
def replace_nan_with_most_frequent_category(df, column):
  most_frequent_category = df[column].mode()[0]
  print(most_frequent_category)
  df[column].fillna(most_frequent_category, inplace = True)
  print(df[column])
  return df

In [ ]:
def remove_missing_values(file_name):
  attribute_dictionary = dict()
  # df = pd.read_csv(file_name, header = None)
  fixed_columns = ['cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
  fixed_columns.insert(0, 'class')
  print(fixed_columns)
  # df.to_csv(file_name, header = fixed_columns, index=False)

  df = pd.read_csv(file_name)

  df = df.replace('?', np.nan)
  # print(df['stalk-root'])
  for column in fixed_columns:
    # print(df[column])
    if df[column].isnull().values.any():
      print(1)
      df = replace_nan_with_most_frequent_category(df, column)

  df.to_csv(file_name, header = fixed_columns, index=False)

  fixed_columns.remove('class')
  # fixed_columns.append('class')

  return df, fixed_columns, attribute_dictionary

In [ ]:
def load_dataset_mushroom(file_name):
  attribute_dictionary = dict()
  df = pd.read_csv(file_name)
  fixed_columns = ['cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']

  # df, fixed_columns, attribute_dictionary = remove_missing_values(file_name)


  labels = df['class'].values.tolist()
  df = df[fixed_columns]
  # print(df)
  # df.to_csv(file_name, index=False)

  for column in fixed_columns:
    attribute_dictionary[column] = df[column].unique()
    # print(len(attribute_dictionary[column]))
  attribute_dictionary['class'] = np.unique(np.array(labels))

  full_table = df.values.tolist()
  fixed_columns.append('class')
  # print(labels)
  return fixed_columns, full_table, labels, attribute_dictionary
  # print(full_table)

In [ ]:
def load_dataset_car(file_name):
  attribute_dictionary = dict()
  df = pd.read_csv(file_name)
  fixed_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
  # df.to_csv(file_name, header = fixed_columns, index=False)

  labels = df['class'].values.tolist()
  fixed_columns.remove('class')
  df = df[fixed_columns]

  for column in fixed_columns:
    attribute_dictionary[column] = df[column].unique()
    # print(len(attribute_dictionary[column]))
  attribute_dictionary['class'] = np.unique(np.array(labels))

  full_table = df.values.tolist()
  fixed_columns.append('class')
  # print(labels)
  return fixed_columns, full_table, labels, attribute_dictionary


In [ ]:
class tree_node:
  def __init__(self, attribute, values, index, majority_class):
    self.attribute = attribute
    self.values = values
    self.dictionary = dict()
    self.label = dict()
    self.child = []
    self.attribute_index = index
    self.majority_class = majority_class

    # i = 0
    # for val in self.values:
    #   self.dictionary[val] = i
    #   i += 1

  def add_child(self, node, label, val, type):
    self.child.append(node)
    cnt = len(self.child)
    self.dictionary[val] = cnt - 1
    # print(self.dictionary)

    if type == "empty":
      self.label[cnt - 1] = label


  # def add_child(self, label, val):
  #   self.child.append(None)
  #   self.label[self.dictionary[val]] = label



In [ ]:
def parse_csv(csv_str, columns, full_table):
  i = 0
  tuples = csv_str.split('\n')
  for s in tuples:
    data = s.split(',')
    data = data[1:]
    print(data)
    if i == 0:
      for d in data:
        columns.append(d)
    elif len(data)!=0:
      full_table.append(data)
    i += 1
  return columns, full_table

In [ ]:
def parse_csv2(csv_str):
  attribute_dictionary = dict()
  tuples = csv_str.split('\n')
  for s in tuples:
    data = s.split(',')
    attribute = data[0]
    if attribute != '' :
      attribute_dictionary[attribute] = data[1:]
  return attribute_dictionary

In [ ]:
def info(database, indices, database_labels, labels):
  answer = 0
  cnt = dict()
  for ind in indices:
    label = database_labels[ind]
    # label = tup[len(tup)-1]
    if label in cnt:
      cnt[label] += 1
    else:
      cnt[label] = 1
  
  for label in labels:
    if label in cnt:
      if cnt[label] == len(indices):
        answer += 0
      else:
        answer += (cnt[label]/len(indices)) * math.log2(cnt[label]/len(indices))

  answer *= -1

  return answer


In [ ]:
def majority_for_no_attribute(columns, database_labels, attribute_dictionary):
  number_of_attributes = len(columns)
  if number_of_attributes == 1:
    values = attribute_dictionary[columns[0]]
    cnt = -1
    label = ""
    for val in values:
      indices = [ind for ind in range(len(database_labels)) if database_labels[ind] == val]

      if len(indices) > cnt:
        cnt = len(indices)
        label = val

    return True, label
  else:
    return False, None

In [ ]:
def majority_class(columns, database_labels, attribute_dictionary):
  label_indices = len(columns) - 1
  values = attribute_dictionary[columns[label_indices]]

  cnt = -1
  label = ""
  for val in values:
    indices = [ind for ind in range(len(database_labels)) if database_labels[ind] == val]

    if len(indices) > cnt:
      cnt = len(indices)
      label = val

  return label

In [ ]:
def create_decision_tree(columns, database, database_labels, attribute_dictionary):
  # print(database)
  number_of_attributes = len(columns)
  min = sys.maxsize
  splitting_attribute = ""
  splitting_attribute_index = -1
    
  
  for i in range(number_of_attributes - 1):
    info_attribute = 0
    flag = False
    values = attribute_dictionary[columns[i]]
    for val in values:
      indices = [ind for ind in range(len(database)) if database[ind][i] == val]
      # print(indices)
      if(len(indices) != 0):
        info_attribute += (len(indices)/len(database)) * info(database, indices, database_labels, attribute_dictionary[columns[number_of_attributes-1]])
        flag = True

    if flag and info_attribute < min:
      min = info_attribute
      splitting_attribute = columns[i]
      splitting_attribute_index = i

  # columns.remove(splitting_attribute)

  new_columns = []

  for i in range(number_of_attributes):
    if i != splitting_attribute_index:
      new_columns.append(columns[i])
  
  # print(splitting_attribute)

  splitting_attribute_values = attribute_dictionary[splitting_attribute]
  label = majority_class(columns, database_labels, attribute_dictionary)
  node = tree_node(splitting_attribute, splitting_attribute_values, splitting_attribute_index, label)

  for val in splitting_attribute_values:
    indices = [ind for ind in range(len(database)) if database[ind][splitting_attribute_index] == val]
    if len(indices) != 0:
      if info(database, indices, database_labels, attribute_dictionary[columns[number_of_attributes-1]]) == 0:
        # print("child found")
        # print(database_labels[indices[0]])
        node.add_child(None, database_labels[indices[0]], val, "empty")
      else:
        new_database = []
        new_database_labels = []

        # for tup,label in zip(database, database_labels):
        #   new_tup = []
        #   # print(tup)
        #   if tup[splitting_attribute_index] == val:
        #     for i in range(len(tup)):
        #       if i != splitting_attribute_index:
        #         new_tup.append(tup[i])
        #     new_database.append(new_tup)
        #     new_database_labels.append()

        for ind in indices:
          new_tup = []
          tup = database[ind]
          for i in range(len(tup)):
            if i != splitting_attribute_index:
              new_tup.append(tup[i])
          new_database.append(new_tup)
          new_database_labels.append(database_labels[ind])

        flag, label_att = majority_for_no_attribute(new_columns, new_database_labels, attribute_dictionary)

        if flag:
          # print(1)
          node.add_child(None, label_att, val, "empty")
        else:
          child_node = create_decision_tree(new_columns, new_database, new_database_labels, attribute_dictionary)
          if child_node == None:
            print("A none node is found")
          node.add_child(child_node, None, val, "")
    else:
      # print(1)
      node.add_child(None, label, val, "empty")
  
  return node



  



    #  = distinct(columns[:][i])

In [ ]:
def print_path(node):
  print(node.attribute)
  print(node.values)

  for i in range(len(node.child)):
    child = node.child[i]
    if child == None:
      print(node.label[i])
    else:
      print_path(child)
    


In [ ]:
def predict(node, test):
  ind = node.attribute_index
  print("values ", node.values)
  val = test[ind]
  print("test value", val)
  val_index = node.dictionary[val]
  print(val_index)
  print("dictionary", node.dictionary)
  print("label", node.label)
  if node.child[val_index] == None:
    if node.label[val_index] == None:
      return node.majority_class
      print("None Object Found")
    else:
      return node.label[val_index]
  else:
    # think about majority voting
    test.pop(ind)
    return predict(node.child[val_index], test)

In [ ]:
# f = open('sample_database.csv', 'r')
# columns, database = parse_csv(f.read(-1), [], [])
# print(columns)
# print(database)
# f = open('attribute_values.csv', 'r')
# attribute_dictionary = parse_csv2(f.read(-1))
# print(attribute_dictionary)
# columns, database, database_labels, attribute_dictionary = load_dataset_mushroom('agaricus-lepiota.data')
num_epochs = 10
accuracy = 0
precision = 0
recall = 0
for i in range(num_epochs):
  columns, database, database_labels, attribute_dictionary = load_dataset_mushroom('agaricus-lepiota.data')
  # columns, database, database_labels, attribute_dictionary = load_dataset_car('car.data')
  X_train, X_test, y_train, y_test = train_test_split(database, database_labels, test_size=0.2)
  print(len(X_train), len(X_test), len(y_train), len(y_test))
  root_node = create_decision_tree(columns, X_train, y_train, attribute_dictionary)
  prediction = []
  for test in X_test:
    # print("size ", len(test))
    # print(test)
    output = predict(root_node, test)
    if output == None :
      # print(test)
      print("None Type object found")
    else:
      prediction.append(output)
  accuracy += accuracy_score(y_test, prediction)
  precision += precision_score(y_test, prediction, average = 'macro')
  recall += recall_score(y_test, prediction, average = 'macro')

print("accuracy ", accuracy/num_epochs)
print("precision ", precision/num_epochs)
print("recall ", recall/num_epochs)
# print(attribute_dictionary)
# for i in range(len(database)):
#   if len(database[i]) < 22:
#     print(i)

Streaming output truncated to the last 5000 lines.
dictionary {'k': 0, 'n': 1, 'u': 2, 'h': 3, 'w': 4, 'r': 5, 'o': 6, 'y': 7, 'b': 8}
label {0: 'e', 1: 'e', 2: 'e', 3: 'e', 5: 'p', 6: 'e', 7: 'e', 8: 'e'}
values  ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
test value n
3
dictionary {'p': 0, 'a': 1, 'l': 2, 'n': 3, 'f': 4, 'c': 5, 'y': 6, 's': 7, 'm': 8}
label {0: 'p', 1: 'e', 2: 'e', 4: 'p', 5: 'p', 6: 'p', 7: 'p', 8: 'p'}
values  ['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']
test value n
1
dictionary {'k': 0, 'n': 1, 'u': 2, 'h': 3, 'w': 4, 'r': 5, 'o': 6, 'y': 7, 'b': 8}
label {0: 'e', 1: 'e', 2: 'e', 3: 'e', 5: 'p', 6: 'e', 7: 'e', 8: 'e'}
values  ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
test value f
4
dictionary {'p': 0, 'a': 1, 'l': 2, 'n': 3, 'f': 4, 'c': 5, 'y': 6, 's': 7, 'm': 8}
label {0: 'p', 1: 'e', 2: 'e', 4: 'p', 5: 'p', 6: 'p', 7: 'p', 8: 'p'}
values  ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
test value f
4
dictionary {'p': 0, 'a': 1, 'l': 2, 'n': 3, 'f': 4, 'c': 5, 'y': 6, 's': 7, 'm': 8}
l